%load_ext autoreload
%autoreload 2

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# load the rm model with the lora adapter applied to the sft model
# make inference on a dataset of choice and get the attributesi  
# swap the attributes back to the prompt, and 
# make inference on the same dataset for generating the output to train a attribute-conditioned model
import json
import gc
import glob
from itertools import chain
import logging
import os
import pathlib
import random
import re
from typing import Callable, Dict, List, Optional, Tuple, Union
import datasets
from dataclasses import dataclass, field
from typing import Optional, List
import accelerate
import pandas as pd
import torch
import tqdm
import transformers
import pandas as pd
import torch

# your PyTorch code here

# clean up the cache
torch.cuda.empty_cache()
import pdb
import json
import gc
import glob
from itertools import chain
import logging
import os
import pathlib
import random
import re
from typing import Callable, Dict, List, Optional, Tuple, Union

import accelerate
import pandas as pd
import torch
import tqdm
import transformers

from peft.utils import WEIGHTS_NAME, get_peft_model_state_dict
from torch.utils.data import DataLoader, TensorDataset

from llava.constants import (
    IGNORE_INDEX,
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
)

from data_utils.data_utils_ppo import QueryResponseDataset

import data_utils.common_utils as common_utils

from data_utils.constants import AnswerType, FACTUAL_PROMPT

import models.rl_models as rl_models

from models.qlora_model import load_4bit_model_for_inference
from models.reward_model import load_4bit_reward_model_for_inference
from models.rl_trainer import (
    AlpacaAccelerator,
    RLTrainer,
    remove_image_token,
    truncate_after_eos_with_padding,
)
from models.ppo_trainer import remove_pad_and_left_pad

from accelerate import DistributedDataParallelKwargs

import data_utils.common_utils as utils
logger = logging.getLogger(__name__)
from llava import conversation as conversation_lib

from peft.utils import WEIGHTS_NAME, get_peft_model_state_dict
try:
    from transformers import LlamaTokenizerFast as LlamaTokenizer

    print("Using fast tokenizer")
except:
    from transformers import LlamaTokenizer

    print("Using slow tokenizer")

from transformers import AutoTokenizer, AutoModelForCausalLM

from llava.constants import (
    IGNORE_INDEX,
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
)

from data_utils.data_utils_ppo import QueryResponseDataset

import data_utils.common_utils as common_utils

from data_utils.constants import AnswerType, FACTUAL_PROMPT

import models.rl_models as rl_models

from models.qlora_model import load_4bit_model_for_inference
from models.reward_model import load_4bit_reward_model_for_inference
from models.rl_trainer import (
    AlpacaAccelerator,
    RLTrainer,
    remove_image_token,
    truncate_after_eos_with_padding,
)
from finetune_lora_ppo import ModelArguments, TrainingArguments, DataArguments
from data_utils.data_utils_ppo import QueryResponseDataset, QueryCoupledResponseDataset
from data_utils.data_utils_ppo import DataCollatorForQueryResponseDataset
class DisableLogger:
    def __enter__(self):
        logging.disable(logging.CRITICAL)

    def __exit__(self, exit_type, exit_value, exit_traceback):
        logging.disable(logging.NOTSET)

rm_path = '/home/ubuntu/RLHF/LLaVA-RLHF-13b-v1.5-336/rm_lora_adapter_model'
rm_model = load_4bit_reward_model_for_inference(rm_path)

vision_tower = rm_model.backbone_model.get_vision_tower()
if not vision_tower.is_loaded:
    vision_tower.load_model()
vision_tower.to(device="cuda", dtype=torch.bfloat16)
vision_tower.requires_grad_(False)
# Tokenizer
tokenizer_model_name = "/home/ubuntu/RLHF/LLaVA-RLHF-13b-v1.5-336/sft_model"
TokenizerClass = AutoTokenizer

tokenizer = TokenizerClass.from_pretrained(
    tokenizer_model_name,
    # cache_dir=args.cache_dir,
    model_max_length=2048, # taken from train_rl_model.sh
    padding_side="left",
    truncation_side="right",
    use_fast=False,
)

tokenizer.pad_token = tokenizer.unk_token
# Dataset loading



model_args = ModelArguments()
training_args = TrainingArguments()
data_args = DataArguments()
data_args.dataset_path = '/home/ubuntu/RLHF/LLaVA-RLHF-Data/llava_ppo50k-aokvqa12k-vqa10k.json'
train_instructions = datasets.load_dataset(
            "json", data_files=data_args.dataset_path
        )
train_df = pd.DataFrame(train_instructions['train'])
# Override the base model name model_args.base_model_name

model_args.base_model_name = tokenizer_model_name

model_args.vision_tower = "openai/clip-vit-large-patch14-336"

if model_args.vision_tower is not None:
    from llava.model import LlavaLlamaForCausalLM

    with DisableLogger():
        base_model = LlavaLlamaForCausalLM.from_pretrained(
            model_args.base_model_name,
            cache_dir=training_args.cache_dir,
        )

    vision_tower = base_model.get_vision_tower()
    if not vision_tower.is_loaded:
        vision_tower.load_model()

    data_args.image_processor = vision_tower.image_processor
    del base_model

if model_args.reward_base_model_name is None:
    model_args.reward_base_model_name = model_args.base_model_name
    data_args.reward_image_processor = vision_tower.image_processor
else:
    with DisableLogger():
        reward_base_model = LlavaLlamaForCausalLM.from_pretrained(
            model_args.reward_base_model_name,
            cache_dir=training_args.cache_dir,
        )
    reward_vision_tower = reward_base_model.get_vision_tower()
    if not reward_vision_tower.is_loaded:
        reward_vision_tower.load_model()
    data_args.reward_image_processor = reward_vision_tower.image_processor
    del reward_base_model

data_args.is_multimodal = True
data_args.mm_use_im_start_end = model_args.mm_use_im_start_end
training_args.use_im_start_end = model_args.mm_use_im_start_end
model_args.mm_vision_select_layer = -2

### A couple critical parameter set borrowed fromt the launch script
data_args.image_aspect_ratio = 'pad'
data_args.image_folder = '/home/ubuntu/latest_llava/llava_1dot5data/coco/train2017'
training_args.query_len = 1280
training_args.response_len = 768
data_args.mm_use_im_start_end = False

############## end of arguments loading

if model_args.version in conversation_lib.conv_templates:
    conversation_lib.default_conversation = conversation_lib.conv_templates[
        model_args.version
    ]
else:
    conversation_lib.default_conversation = conversation_lib.conv_templates[
        "vicuna_v1"
    ]
train_dataset = QueryCoupledResponseDataset(
    df=train_df,
    tokenizer=tokenizer,
    query_len=training_args.query_len,
    response_len=training_args.response_len,
    data_args=data_args,
)
train_dataset.query_attn_masks = train_dataset.queries.ne(tokenizer.pad_token_id).long()

make_rl_data_module_output_dict = dict(
        train_dataset=train_dataset,
        eval_dataset=None,
        data_collator=DataCollatorForQueryResponseDataset(),
    )

### Definition the inference class

class GetAttributeModel:
    def __init__(self, rm_model, tokenizer, accelerator: AlpacaAccelerator, training_args) -> None:
        self.reward_model = rm_model
        self.tokenizer = tokenizer
        self.accelerator = accelerator
        self.training_args = training_args
        self.args = training_args # TODO add other args
        self.args.reward_prompt_file = "/home/ubuntu/RLHF/LLaVA-RLHF/RLHF/scripts/13b-v1.5-336/train_reward_model.sh"
        self.train_dataset = make_rl_data_module_output_dict['train_dataset']
        self.data_collator = make_rl_data_module_output_dict['data_collator']


        self.reward_model_prompt = None
        self.reward_model_prompt_untokenized = None

        if self.args.reward_prompt_file is not None:
            with open(self.args.reward_prompt_file, "r") as f:
                self.reward_model_prompt_untokenized = " " + f.read().strip()
            self.reward_model_prompt = self.tokenizer.encode(
                self.reward_model_prompt_untokenized,
                return_tensors="pt",
                add_special_tokens=False,
            )
        self.image_to_caption_mapping = None

    def get_train_dataloader(self):
        logger.warning(
            f"Train dataset size: {len(self.train_dataset)}",
            # main_process_only=True
        )  # noqa
        train_dataloader = DataLoader(
            dataset=self.train_dataset,
            collate_fn=self.data_collator,
            batch_size=self.training_args.rollout_per_device_batch_size,
            shuffle=True,
            drop_last=True,
        )
        train_dataloader = self.accelerator.prepare(train_dataloader)  # noqa
        # self._log_batch_size(train_dataloader, "train_dataloader")
        return utils.InfiniteLoader(train_dataloader)

    def step(self, train_dataloader, step_idx: int):
        # TODO fix the range
        queries_batches = [
            next(train_dataloader) for _ in range(1)
        ]
        results = []
        rollouts = self.rollout(queries_batches)
        return rollouts


    @torch.inference_mode()
    def rollout(self, queries_data) -> Dict[str, torch.Tensor]:

        """Rollout trajectories with policy.

        Args:
            queries_data: Sequence of batches or DataLoader.
                Each batch is a dict with keys 'queries' and 'query_attn_masks' and 'response'.

        Returns:
            Dictionary with keys
                'queries', 'query_attn_masks', 'responses',
                'logprobs', 'ref_logprobs', 'values',
                'rewards', 'non_score_rewards', 'shaped_rewards'.
        """
        # Give up dropout throughout.
        # self.policy.eval()
        # `keep_fp32_wrapper` retains the autocast wrapper of model.forward created by accelerate:
        #  recall one sets mixed precision options with accelerator.
        # The precise value of this arg doesn't matter here, since we use the unwrapped model only for respond.
        # Generally, try to use the wrapped model as much as you can, since it's got the autocast/cast-back wrappers.


        self.reward_model.eval()

        rollouts = []
        for batch_idx, batch in tqdm.tqdm(
            enumerate(queries_data),
            total=len(queries_data),
            disable=not self.accelerator.is_main_process,
            desc="rollout",
        ):
            gc.collect()
            torch.cuda.empty_cache()
            # Sample rollouts.
            (
                images,
                reward_images,
                image_file_ids,
                caption_types,
                length_bonus_multiplier,
                queries,
                query_attn_masks,
                responses,
            ) = common_utils.unpack_dict(
                common_utils.prepare_inputs(batch, device=self.accelerator.device),
                keys=(
                    "images",
                    "reward_images",
                    "image_file_ids",
                    "caption_types",
                    "length_bonus_multiplier",
                    "queries",
                    "query_attn_masks",
                    "responses"
                ),
            )

            if self.args.bf16:
                images = images.to(torch.bfloat16)
                reward_images = reward_images.to(torch.bfloat16)
            elif self.args.fp16:
                images = images.half()
                reward_images = reward_images.half()
            # TODO: replace with the response from the dataset, not model generated
            # respond_outputs = unwrapped_policy.respond(
            #     queries, query_attn_masks, images, temperature=self.args.temperature
            # )
            # (responses,) = common_utils.unpack_dict(respond_outputs, ("responses",))

            additional_token1 = self.tokenizer.encode("?", add_special_tokens=False)[0]
            assert additional_token1 == 1577

            additional_token2 = self.tokenizer.encode("\n?")[-1]
            assert additional_token2 == 29973

            responses = truncate_after_eos_with_padding(
                responses,
                self.tokenizer.eos_token_id,
                self.tokenizer.pad_token_id,
                additional_tokens=[additional_token1, additional_token2],
            )

            rollouts_batch = {
                "images": images,
                "reward_images": reward_images,
                "queries": queries,
                "query_attn_masks": query_attn_masks,
                "responses": responses,
            }
            # Decode the response 
            text_responses = self.tokenizer.batch_decode(
                responses,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=False,
            )

            correct_bonus = []
            # go through the text responses and assign bonus (optional)
            # for idx, response in enumerate(text_responses):
            #     caption_type = AnswerType(caption_types[idx].item())

            #     if caption_type == AnswerType.GENERAL:
            #         correct_bonus.append(0.0)
            #     elif caption_type in [AnswerType.A_IN_ABCD, AnswerType.B_IN_ABCD, AnswerType.C_IN_ABCD, AnswerType.D_IN_ABCD]:
            #         expected_start = caption_type.name.split("_")[0] + "."
            #         expected_phrase = "correct option is " + expected_start
            #         if response.strip().startswith(expected_start) or expected_phrase in response:
            #             correct_bonus.append(1.0)
            #         else:
            #             correct_bonus.append(0.0)
            #     elif caption_type == AnswerType.NO_IN_YESNO:
            #         if response.strip().startswith("No"):
            #             correct_bonus.append(0.5)
            #         elif response.strip().startswith("Yes"):
            #             correct_bonus.append(-0.5)
            #         else:
            #             correct_bonus.append(0.0)
            #     elif caption_type == AnswerType.YES_IN_YESNO:
            #         # TODO(zhiqings): for now, we do not give symbolic award for "Yes" in Yes/No questions.
            #         correct_bonus.append(0.0)
            #     else:
            #         raise NotImplementedError
            # assert len(correct_bonus) == len(text_responses)
            # correct_bonus = torch.tensor(correct_bonus, device=responses.device)

            has_stop_token = [
                self.tokenizer.eos_token_id in response
                for response in responses.tolist()
            ]

            sequences = [
                torch.concat((query, response), dim=0)
                for query, response in zip(queries, responses)
            ]
            sequences = torch.stack(sequences, dim=0)

            sequences = remove_pad_and_left_pad(
                sequences,
                pad_token_id=self.tokenizer.pad_token_id,
            )
            # Prepareing the reward model prompt, 
            # may have fancy way to add captions or context to the prompt, dubbed as FACTUAL-RLHF in LLaVa-RLHF paper
            if self.reward_model_prompt is not None:
                if self.image_to_caption_mapping is None:
                    reward_model_prompt = (
                        self.reward_model_prompt.reshape(1, -1)
                        .repeat(len(sequences), 1)
                        .to(self.accelerator.device)
                    )
                    sequences = torch.cat((sequences, reward_model_prompt), dim=1)
                else:
                    reward_model_prompt_untokenized = (
                        self.reward_model_prompt_untokenized
                    )
                    image_to_caption_mapping = self.image_to_caption_mapping

                    image_ids = []
                    for i in range(len(sequences)):
                        image_file = str(image_file_ids[i].item()).zfill(12) + ".jpg"
                        caption_type = AnswerType(caption_types[i].item())
                        if caption_type in [AnswerType.GENERAL, AnswerType.NO_IN_YESNO, AnswerType.YES_IN_YESNO]:
                            image_id = image_file
                        elif caption_type in [AnswerType.A_IN_ABCD, AnswerType.B_IN_ABCD, AnswerType.C_IN_ABCD, AnswerType.D_IN_ABCD]:
                            image_id = "aok_" + image_file
                        else:
                            print(caption_type)
                            print([AnswerType.GENERAL, AnswerType.NO_IN_YESNO, AnswerType.YES_IN_YESNO])
                            print([AnswerType.A_IN_ABCD, AnswerType.B_IN_ABCD, AnswerType.C_IN_ABCD, AnswerType.D_IN_ABCD])
                            raise NotImplementedError
                        image_ids.append(image_id)

                    captions = [
                        image_to_caption_mapping[image_id] for image_id in image_ids
                    ]

                    assert r"{factual_prompt}" in reward_model_prompt_untokenized

                    reward_model_prompts = []

                    for caption_list in captions:
                        caption_list = caption_list[:]
                        random.shuffle(caption_list)
                        factual_prompt = FACTUAL_PROMPT
                        for caption in caption_list:
                            factual_prompt = factual_prompt + f"  - {caption}\n"
                        reward_model_prompt_per_example = (
                            reward_model_prompt_untokenized.format(
                                factual_prompt=factual_prompt
                            )
                        )
                        reward_model_prompts.append(reward_model_prompt_per_example)
                    reward_model_prompts = self.tokenizer(
                        reward_model_prompts,
                        return_tensors="pt",
                        add_special_tokens=False,
                        padding="longest",
                    )["input_ids"]
                    reward_model_prompts = reward_model_prompts.to(
                        self.accelerator.device
                    )

                    sequences = torch.cat((sequences, reward_model_prompts), dim=1)
                    sequences = remove_pad_and_left_pad(
                        sequences,
                        pad_token_id=self.tokenizer.pad_token_id,
                    )

            clean_sequences = sequences.detach().clone()
            clean_sequences[
                clean_sequences == IMAGE_TOKEN_INDEX
            ] = self.tokenizer.eos_token_id

            text_sequences = self.tokenizer.batch_decode(
                clean_sequences,
                skip_special_tokens=False,
                clean_up_tokenization_spaces=False,
            )

            if self.accelerator.is_main_process:
                print("=" * 20)
                print(text_sequences[0].split("<unk><s> ")[-1])
                print("=" * 20)
                image_id = image_file_ids[0].item()
                # convert int into "000000xxxxxx.jpg"
                image_id = (
                    "https://s3.us-east-1.amazonaws.com/images.cocodataset.org/train2017/"
                    + str(image_id).zfill(12)
                    + ".jpg"
                )
                print(image_id)
                print("=" * 20)
            # OPTIONAL: compute the length bonus 
            non_pad_mask = responses.ne(self.tokenizer.pad_token_id)
            non_pad_seq_len = (
                non_pad_mask.sum(dim=1).float().to(self.accelerator.device)
            )
            length_bonus = non_pad_seq_len / float(self.args.response_len)

            # convert length_bonus_multiplier to the shape, type, and device of length_bonus
            length_bonus = length_bonus * length_bonus_multiplier.to(
                length_bonus.device
            ).reshape(length_bonus.shape).to(length_bonus.dtype)

            sequences_attention_mask = sequences.ne(self.tokenizer.pad_token_id)

            # Evaluate logprobs of the samples.



            rollouts_batch["length_bonus"] = length_bonus
            rollouts_batch["correct_bonus"] = correct_bonus
            sub_batch_size = self.args.reward_model_per_device_batch_size
            batch_size_per_device = rollouts_batch["responses"].shape[0]
            if sub_batch_size is None or sub_batch_size == batch_size_per_device:
                reward_outputs = self.reward_model(
                    input_ids=sequences,
                    attention_mask=sequences_attention_mask,
                    images=reward_images,
                )
                print(reward_outputs)
            else:
                assert batch_size_per_device % sub_batch_size == 0

                reward_outputs_list = []

                for sub_batch_idx in range(batch_size_per_device // sub_batch_size):
                    idx_start = sub_batch_idx * sub_batch_size
                    idx_end = (sub_batch_idx + 1) * sub_batch_size
                    sub_batch_reward_outputs = self.reward_model(
                        input_ids=sequences[idx_start:idx_end],
                        attention_mask=sequences_attention_mask[idx_start:idx_end],
                        images=reward_images[idx_start:idx_end],
                    )
                    reward_outputs_list.append(sub_batch_reward_outputs)

                reward_outputs = common_utils.merge_dict(
                    reward_outputs_list, merge_fn=torch.cat
                )
                del reward_outputs_list
                del sub_batch_reward_outputs
            # Remove the penality for sequences that did not stop properly
            # reward_outputs = self.post_reward(
            #     reward_outputs,
            #     responses,
            #     penalize_no_stop_token=self.args.penalize_no_stop_token,
            #     relative_stop_token_penalty=self.args.relative_stop_token_penalty,
            #     has_stop_token=has_stop_token,
            # )
            rollouts_batch.update(reward_outputs)
            print(f'rollouts_batch: {rollouts_batch}')

            # Shape reward with KL penalty.
            # shape_reward_outputs = self._shape_reward(
            #     rewards=rollouts_batch["rewards"],
            #     responses=rollouts_batch["responses"],
            #     logprobs=rollouts_batch["logprobs"],
            #     ref_logprobs=rollouts_batch["ref_logprobs"],
            #     length_bonus=rollouts_batch["length_bonus"],
            #     correct_bonus=rollouts_batch["correct_bonus"],
            # )
            rollouts_batch_cpu = {
                key: value for key, value in rollouts_batch.items()
            }
            rollouts.append(rollouts_batch_cpu)

        # # Items in dict need to be of same shape.
        # rollouts = common_utils.merge_dict(rollouts, merge_fn=torch.cat)

        # # Estimating advantages outside the loop gives more samples for reward normalization.
        # advantages = self._estimate_advantage(
        #     rewards=rollouts["shaped_rewards"].to(self.accelerator.device),
        #     values=rollouts["values"].to(self.accelerator.device),
        # )
        # advantages = {key: value.cpu() for key, value in advantages.items()}
        # pdb.set_trace()
        return rollouts_batch
    
############ End of definition of the inference task ###############


/home/ubuntu/mambaforge-pypy3/envs/llavav2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-10-27 05:05:03,152] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Using fast tokenizer
Using fast tokenizer
loading base model /home/ubuntu/RLHF/LLaVA-RLHF-13b-v1.5-336/sft_model...


Loading checkpoint shards: 100%|██████████| 3/3 [01:49<00:00, 36.36s/it]
Some weights of the model checkpoint at /home/ubuntu/RLHF/LLaVA-RLHF-13b-v1.5-336/sft_model were not used when initializing LlavaLlamaForCausalLM: ['model.vision_tower.vision_tower.vision_model.encoder.layers.16.self_attn.v_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.5.self_attn.out_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.21.mlp.fc1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.15.self_attn.v_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.23.self_attn.q_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.7.layer_norm1.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.23.self_attn.v_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.16.layer_norm1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.1.self_attn.k_proj.bias', '

Loading adapters from checkpoint.


Rollout stats:
	rollout_batch_size: 512
	rollout_per_device_batch_size: 32
	world_size: 1

Step stats:
	step_batch_size: 256
	step_per_device_batch_size: 2
	world_size: 1

Accumulation steps:
	rollout_accumulation_steps: 16
	gradient_accumulation_steps: 128

Found cached dataset json (/home/ubuntu/.cache/huggingface/datasets/json/default-226d9145801c0b24/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|██████████| 72056/72056 [02:06<00:00, 568.64it/s]
/home/ubuntu/RLHF/LLaVA-RLHF/RLHF/data_utils/data_utils_ppo.py:267: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(query, dtype=torch.long).view(-1)[:-3] for query in queries
Max query length: 554
Filtered out 0 instances out of 72056 that exceed length limit. These examples are not used for training, but will still be used in evaluation. 
100%|█████

In [3]:
accelerator = AlpacaAccelerator(
log_with=training_args.report_to,
project_dir=training_args.logging_dir,
gradient_accumulation_steps=training_args.gradient_accumulation_steps,
even_batches=True,  # Make sure the batch size on each device is the same.
split_batches=False,  # Don't break a batch into smaller chunks.
step_scheduler_with_optimizer=False,  # Untie optimizer and scheduler step.
# Value model might not use all parameters (e.g., lm-head) in the forward pass.
kwargs_handlers=[
    DistributedDataParallelKwargs(
        find_unused_parameters=training_args.ddp_find_unused_parameters,
    )
],
)

attribute_model = GetAttributeModel(rm_model, tokenizer, accelerator, training_args)
attribute_model.training_args.rollout_per_device_batch_size = 16
train_dataloader = attribute_model.get_train_dataloader()
for i in range(1):
    rollout = attribute_model.step(train_dataloader, i)
    print(rollout.keys())

/home/ubuntu/mambaforge-pypy3/envs/llavav2/lib/python3.10/site-packages/accelerate/accelerator.py:371: UserWarning: `log_with=[]` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")
Train dataset size: 72056


In [ ]:
rollout

# Legacy code


In [2]:
# load the rm model with the lora adapter applied to the sft model
# make inference on a dataset of choice and get the attributesi  
# swap the attributes back to the prompt, and 
# make inference on the same dataset for generating the output to train a attribute-conditioned model
import json
import gc
import glob
from itertools import chain
import logging
import os
import pathlib
import random
import re
from typing import Callable, Dict, List, Optional, Tuple, Union

import accelerate
import pandas as pd
import torch
import tqdm
import transformers

from peft.utils import WEIGHTS_NAME, get_peft_model_state_dict

from llava.constants import (
    IGNORE_INDEX,
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
)

from data_utils.data_utils_ppo import QueryResponseDataset

import data_utils.common_utils as common_utils

from data_utils.constants import AnswerType, FACTUAL_PROMPT

import models.rl_models as rl_models

from models.qlora_model import load_4bit_model_for_inference
from models.reward_model import load_4bit_reward_model_for_inference
from models.rl_trainer import (
    AlpacaAccelerator,
    RLTrainer,
    remove_image_token,
    truncate_after_eos_with_padding,
)
rm_path = '/home/ubuntu/RLHF/LLaVA-RLHF-13b-v1.5-336/rm_lora_adapter_model'
rm_model = load_4bit_reward_model_for_inference(rm_path)


/home/ubuntu/mambaforge-pypy3/envs/llavav2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-10-25 23:33:18,660] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
loading base model /home/ubuntu/RLHF/LLaVA-RLHF-13b-v1.5-336/sft_model...


Loading checkpoint shards: 100%|██████████| 3/3 [01:53<00:00, 37.71s/it]
Some weights of the model checkpoint at /home/ubuntu/RLHF/LLaVA-RLHF-13b-v1.5-336/sft_model were not used when initializing LlavaLlamaForCausalLM: ['model.vision_tower.vision_tower.vision_model.encoder.layers.13.self_attn.q_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.6.layer_norm2.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.8.mlp.fc2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.18.layer_norm2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.23.mlp.fc2.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.14.self_attn.v_proj.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.11.self_attn.v_proj.bias', 'model.vision_tower.vision_tower.vision_model.encoder.layers.14.mlp.fc1.weight', 'model.vision_tower.vision_tower.vision_model.encoder.layers.11.self_attn.k_proj.weight', 'model.vision_to

Loading adapters from checkpoint.


In [38]:
vision_tower = rm_model.backbone_model.get_vision_tower()
if not vision_tower.is_loaded:
    vision_tower.load_model()
vision_tower.to(device="cuda", dtype=torch.bfloat16)
vision_tower.requires_grad_(False)

CLIPVisionTower(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): QuickGELUActivation

In [4]:
try:
    from transformers import LlamaTokenizerFast as LlamaTokenizer

    print("Using fast tokenizer")
except:
    from transformers import LlamaTokenizer

    print("Using slow tokenizer")

from transformers import AutoTokenizer, AutoModelForCausalLM



Using fast tokenizer


In [5]:
# Tokenizer loading

tokenizer_model_name = "/home/ubuntu/RLHF/LLaVA-RLHF-13b-v1.5-336/sft_model"
TokenizerClass = AutoTokenizer

tokenizer = TokenizerClass.from_pretrained(
    tokenizer_model_name,
    # cache_dir=args.cache_dir,
    model_max_length=2048, # taken from train_rl_model.sh
    padding_side="left",
    truncation_side="right",
    use_fast=False,
)

tokenizer.pad_token = tokenizer.unk_token

# 2. lets see the RM model applied to the RLHF data

RLHF data is made of 



The PPO training of the policy model is based on the prompt combination of:

Our RL split of the LLaVA data (50k)
A-OKVQA in the CoT format (12k)
Yes/No Questions from VQA-v2 (10k)

present in the LLaVA-RLHF-Data/llava_ppo50k-aokvqa12k-vqa10k.json file. 


partially implemented in the rollout method in ppo_trainer (torch inference mode def rollout)

Line 181 until 550 without the KL penality stuff


lets say we have rewards per sample, a float applied. 





### Dataset loading


In [6]:
import datasets
from dataclasses import dataclass, field
from typing import Optional, List
@dataclass
class DataArguments:
    dataset_path: str = field(default="tatsu-lab/alpaca_farm")
    dataset_name: str = field(default="alpaca_instructions")
    train_splits: List[str] = field(default_factory=lambda: ["unlabeled"])
    stop_token: Optional[str] = field(
        default=None,
        metadata={"help": "Token to stop generation with."},
    )
    # From LLaVA
    lazy_preprocess: bool = False
    is_multimodal: bool = False
    image_folder: Optional[str] = field(default=None)
    image_aspect_ratio: str = "square"
    image_grid_pinpoints: Optional[str] = field(default=None)

DataArguments.dataset_path = '/home/ubuntu/RLHF/LLaVA-RLHF-Data/llava_ppo50k-aokvqa12k-vqa10k.json'
train_instructions = datasets.load_dataset(
            "json", data_files=DataArguments.dataset_path
        )

Found cached dataset json (/home/ubuntu/.cache/huggingface/datasets/json/default-226d9145801c0b24/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)
100%|██████████| 1/1 [00:00<00:00,  1.37it/s]


The following is takenfrom data_utils_ppo and make_rl_data_module

In [7]:
import pandas as pd
train_df = pd.DataFrame(train_instructions['train'])

In [8]:
train_df.iloc[0]['conversations']

[{'from': 'human',
  'value': "<image>\nWhat activity could develop the young girl's physical and cognitive abilities?"},
 {'from': 'gpt',
  'value': "Flying a kite, like in the image, can be a fun activity that helps develop a young girl's physical and cognitive abilities. This activity encourages physical movement, such as running in open spaces, and helps improve hand-eye coordination as the child navigates the kite in the sky. Additionally, flying a kite requires problem-solving and strategic thinking, as the child must understand wind patterns and make adjustments to maintain the kite's flight. Overall, kite flying not only serves as a recreational activity but also contributes to the child's growth and development."}]

####  The difference is that in RL, the response is sampled from the policy (model) whereas here I am making inferennce given (query, response) to a score giveen by the reward model

In [9]:

class DisableLogger:
    def __enter__(self):
        logging.disable(logging.CRITICAL)

    def __exit__(self, exit_type, exit_value, exit_traceback):
        logging.disable(logging.NOTSET)

In [10]:
from finetune_lora_ppo import ModelArguments, TrainingArguments, DataArguments
model_args = ModelArguments()
training_args = TrainingArguments()
data_args = DataArguments()
# Override the base model name model_args.base_model_name

model_args.base_model_name = tokenizer_model_name

VISION_TOWER="openai/clip-vit-large-patch14-336"
model_args.vision_tower = VISION_TOWER

if model_args.vision_tower is not None:
    from llava.model import LlavaLlamaForCausalLM

    with DisableLogger():
        base_model = LlavaLlamaForCausalLM.from_pretrained(
            model_args.base_model_name,
            cache_dir=training_args.cache_dir,
        )

    vision_tower = base_model.get_vision_tower()
    if not vision_tower.is_loaded:
        vision_tower.load_model()

    data_args.image_processor = vision_tower.image_processor
    del base_model

Rollout stats:
	rollout_batch_size: 512
	rollout_per_device_batch_size: 32
	world_size: 1

Step stats:
	step_batch_size: 256
	step_per_device_batch_size: 2
	world_size: 1

Accumulation steps:
	rollout_accumulation_steps: 16
	gradient_accumulation_steps: 128



Using fast tokenizer


Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.27s/it]


In [12]:
if model_args.reward_base_model_name is None:
    model_args.reward_base_model_name = model_args.base_model_name
    data_args.reward_image_processor = vision_tower.image_processor
else:
    with DisableLogger():
        reward_base_model = LlavaLlamaForCausalLM.from_pretrained(
            model_args.reward_base_model_name,
            cache_dir=training_args.cache_dir,
        )
    reward_vision_tower = reward_base_model.get_vision_tower()
    if not reward_vision_tower.is_loaded:
        reward_vision_tower.load_model()
    data_args.reward_image_processor = reward_vision_tower.image_processor
    del reward_base_model

data_args.is_multimodal = True
data_args.mm_use_im_start_end = model_args.mm_use_im_start_end
training_args.use_im_start_end = model_args.mm_use_im_start_end
model_args.mm_vision_select_layer = -2

### A couple critical parameter set borrowed fromt the launch script
data_args.image_aspect_ratio = 'pad'
data_args.image_folder = '/home/ubuntu/latest_llava/llava_1dot5data/coco/train2017'


Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.01s/it]


In [13]:
from data_utils.data_utils_ppo import QueryResponseDataset, QueryCoupledResponseDataset


In [14]:
training_args.query_len = 1280
training_args.response_len = 768
data_args.mm_use_im_start_end = False

In [15]:
from data_utils.data_utils_ppo import QueryResponseDataset, QueryCoupledResponseDataset
from llava import conversation as conversation_lib

if model_args.version in conversation_lib.conv_templates:
    conversation_lib.default_conversation = conversation_lib.conv_templates[
        model_args.version
    ]
else:
    conversation_lib.default_conversation = conversation_lib.conv_templates[
        "vicuna_v1"
    ]
train_dataset = QueryCoupledResponseDataset(
    df=train_df,
    tokenizer=tokenizer,
    query_len=training_args.query_len,
    response_len=training_args.response_len,
    data_args=data_args,
)

100%|██████████| 72056/72056 [02:06<00:00, 569.17it/s]
/home/ubuntu/RLHF/LLaVA-RLHF/RLHF/data_utils/data_utils_ppo.py:267: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(query, dtype=torch.long).view(-1)[:-3] for query in queries
Max query length: 554
Filtered out 0 instances out of 72056 that exceed length limit. These examples are not used for training, but will still be used in evaluation. 
100%|██████████| 72056/72056 [01:46<00:00, 674.64it/s]
/home/ubuntu/RLHF/LLaVA-RLHF/RLHF/data_utils/data_utils_ppo.py:320: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(response, dtype=torch.long).view(-1)[:-3] for response in responses
Max query length: 535
Filtered out 0 

In [17]:
train_dataset.query_attn_masks = train_dataset.queries.ne(tokenizer.pad_token_id).long()

In [18]:
train_dataset[0]['responses']

tensor([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

### A couple critical parameter set borrowed fromt the launch script

In [19]:
data_args.image_aspect_ratio = 'pad'
data_args.image_folder = '/home/ubuntu/latest_llava/llava_1dot5data/coco/train2017'

# TODO fix the length (query_len) 

In [20]:
from data_utils.data_utils_ppo import DataCollatorForQueryResponseDataset
make_rl_data_module_output_dict = dict(
        train_dataset=train_dataset,
        eval_dataset=None,
        data_collator=DataCollatorForQueryResponseDataset(),
    )

In [69]:

import torch

# your PyTorch code here

# clean up the cache
torch.cuda.empty_cache()
import pdb
import json
import gc
import glob
from itertools import chain
import logging
import os
import pathlib
import random
import re
from typing import Callable, Dict, List, Optional, Tuple, Union

import accelerate
import pandas as pd
import torch
import tqdm
import transformers

from peft.utils import WEIGHTS_NAME, get_peft_model_state_dict
from torch.utils.data import DataLoader, TensorDataset

from llava.constants import (
    IGNORE_INDEX,
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
)

from data_utils.data_utils_ppo import QueryResponseDataset

import data_utils.common_utils as common_utils

from data_utils.constants import AnswerType, FACTUAL_PROMPT

import models.rl_models as rl_models

from models.qlora_model import load_4bit_model_for_inference
from models.reward_model import load_4bit_reward_model_for_inference
from models.rl_trainer import (
    AlpacaAccelerator,
    RLTrainer,
    remove_image_token,
    truncate_after_eos_with_padding,
)
from models.ppo_trainer import remove_pad_and_left_pad

from accelerate import DistributedDataParallelKwargs

import data_utils.common_utils as utils
logger = logging.getLogger(__name__)

class GetAttributeModel:
    def __init__(self, rm_model, tokenizer, accelerator: AlpacaAccelerator, training_args) -> None:
        self.reward_model = rm_model
        self.tokenizer = tokenizer
        self.accelerator = accelerator
        self.training_args = training_args
        self.args = training_args # TODO add other args
        self.args.reward_prompt_file = "/home/ubuntu/RLHF/LLaVA-RLHF/RLHF/scripts/13b-v1.5-336/train_reward_model.sh"
        self.train_dataset = make_rl_data_module_output_dict['train_dataset']
        self.data_collator = make_rl_data_module_output_dict['data_collator']


        self.reward_model_prompt = None
        self.reward_model_prompt_untokenized = None

        if self.args.reward_prompt_file is not None:
            with open(self.args.reward_prompt_file, "r") as f:
                self.reward_model_prompt_untokenized = " " + f.read().strip()
            self.reward_model_prompt = self.tokenizer.encode(
                self.reward_model_prompt_untokenized,
                return_tensors="pt",
                add_special_tokens=False,
            )
        self.image_to_caption_mapping = None

    def get_train_dataloader(self):
        logger.warning(
            f"Train dataset size: {len(self.train_dataset)}",
            # main_process_only=True
        )  # noqa
        train_dataloader = DataLoader(
            dataset=self.train_dataset,
            collate_fn=self.data_collator,
            batch_size=self.training_args.rollout_per_device_batch_size,
            shuffle=True,
            drop_last=True,
        )
        train_dataloader = self.accelerator.prepare(train_dataloader)  # noqa
        # self._log_batch_size(train_dataloader, "train_dataloader")
        return utils.InfiniteLoader(train_dataloader)

    def step(self, train_dataloader, step_idx: int):
        # TODO fix the range
        queries_batches = [
            next(train_dataloader) for _ in range(1)
        ]
        results = []
        rollouts = self.rollout(queries_batches)
        return rollouts


    @torch.inference_mode()
    def rollout(self, queries_data) -> Dict[str, torch.Tensor]:

        """Rollout trajectories with policy.

        Args:
            queries_data: Sequence of batches or DataLoader.
                Each batch is a dict with keys 'queries' and 'query_attn_masks' and 'response'.

        Returns:
            Dictionary with keys
                'queries', 'query_attn_masks', 'responses',
                'logprobs', 'ref_logprobs', 'values',
                'rewards', 'non_score_rewards', 'shaped_rewards'.
        """
        # Give up dropout throughout.
        # self.policy.eval()
        # `keep_fp32_wrapper` retains the autocast wrapper of model.forward created by accelerate:
        #  recall one sets mixed precision options with accelerator.
        # The precise value of this arg doesn't matter here, since we use the unwrapped model only for respond.
        # Generally, try to use the wrapped model as much as you can, since it's got the autocast/cast-back wrappers.


        self.reward_model.eval()

        rollouts = []
        for batch_idx, batch in tqdm.tqdm(
            enumerate(queries_data),
            total=len(queries_data),
            disable=not self.accelerator.is_main_process,
            desc="rollout",
        ):
            gc.collect()
            torch.cuda.empty_cache()
            # Sample rollouts.
            (
                images,
                reward_images,
                image_file_ids,
                caption_types,
                length_bonus_multiplier,
                queries,
                query_attn_masks,
                responses,
            ) = common_utils.unpack_dict(
                common_utils.prepare_inputs(batch, device=self.accelerator.device),
                keys=(
                    "images",
                    "reward_images",
                    "image_file_ids",
                    "caption_types",
                    "length_bonus_multiplier",
                    "queries",
                    "query_attn_masks",
                    "responses"
                ),
            )

            if self.args.bf16:
                images = images.to(torch.bfloat16)
                reward_images = reward_images.to(torch.bfloat16)
            elif self.args.fp16:
                images = images.half()
                reward_images = reward_images.half()
            # TODO: replace with the response from the dataset, not model generated
            # respond_outputs = unwrapped_policy.respond(
            #     queries, query_attn_masks, images, temperature=self.args.temperature
            # )
            # (responses,) = common_utils.unpack_dict(respond_outputs, ("responses",))

            additional_token1 = self.tokenizer.encode("?", add_special_tokens=False)[0]
            assert additional_token1 == 1577

            additional_token2 = self.tokenizer.encode("\n?")[-1]
            assert additional_token2 == 29973

            responses = truncate_after_eos_with_padding(
                responses,
                self.tokenizer.eos_token_id,
                self.tokenizer.pad_token_id,
                additional_tokens=[additional_token1, additional_token2],
            )

            rollouts_batch = {
                "images": images,
                "reward_images": reward_images,
                "queries": queries,
                "query_attn_masks": query_attn_masks,
                "responses": responses,
            }
            # Decode the response 
            text_responses = self.tokenizer.batch_decode(
                responses,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=False,
            )

            correct_bonus = []
            # go through the text responses and assign bonus (optional)
            # for idx, response in enumerate(text_responses):
            #     caption_type = AnswerType(caption_types[idx].item())

            #     if caption_type == AnswerType.GENERAL:
            #         correct_bonus.append(0.0)
            #     elif caption_type in [AnswerType.A_IN_ABCD, AnswerType.B_IN_ABCD, AnswerType.C_IN_ABCD, AnswerType.D_IN_ABCD]:
            #         expected_start = caption_type.name.split("_")[0] + "."
            #         expected_phrase = "correct option is " + expected_start
            #         if response.strip().startswith(expected_start) or expected_phrase in response:
            #             correct_bonus.append(1.0)
            #         else:
            #             correct_bonus.append(0.0)
            #     elif caption_type == AnswerType.NO_IN_YESNO:
            #         if response.strip().startswith("No"):
            #             correct_bonus.append(0.5)
            #         elif response.strip().startswith("Yes"):
            #             correct_bonus.append(-0.5)
            #         else:
            #             correct_bonus.append(0.0)
            #     elif caption_type == AnswerType.YES_IN_YESNO:
            #         # TODO(zhiqings): for now, we do not give symbolic award for "Yes" in Yes/No questions.
            #         correct_bonus.append(0.0)
            #     else:
            #         raise NotImplementedError
            # assert len(correct_bonus) == len(text_responses)
            # correct_bonus = torch.tensor(correct_bonus, device=responses.device)

            has_stop_token = [
                self.tokenizer.eos_token_id in response
                for response in responses.tolist()
            ]

            sequences = [
                torch.concat((query, response), dim=0)
                for query, response in zip(queries, responses)
            ]
            sequences = torch.stack(sequences, dim=0)

            sequences = remove_pad_and_left_pad(
                sequences,
                pad_token_id=self.tokenizer.pad_token_id,
            )
            # Prepareing the reward model prompt, 
            # may have fancy way to add captions or context to the prompt, dubbed as FACTUAL-RLHF in LLaVa-RLHF paper
            if self.reward_model_prompt is not None:
                if self.image_to_caption_mapping is None:
                    reward_model_prompt = (
                        self.reward_model_prompt.reshape(1, -1)
                        .repeat(len(sequences), 1)
                        .to(self.accelerator.device)
                    )
                    sequences = torch.cat((sequences, reward_model_prompt), dim=1)
                else:
                    reward_model_prompt_untokenized = (
                        self.reward_model_prompt_untokenized
                    )
                    image_to_caption_mapping = self.image_to_caption_mapping

                    image_ids = []
                    for i in range(len(sequences)):
                        image_file = str(image_file_ids[i].item()).zfill(12) + ".jpg"
                        caption_type = AnswerType(caption_types[i].item())
                        if caption_type in [AnswerType.GENERAL, AnswerType.NO_IN_YESNO, AnswerType.YES_IN_YESNO]:
                            image_id = image_file
                        elif caption_type in [AnswerType.A_IN_ABCD, AnswerType.B_IN_ABCD, AnswerType.C_IN_ABCD, AnswerType.D_IN_ABCD]:
                            image_id = "aok_" + image_file
                        else:
                            print(caption_type)
                            print([AnswerType.GENERAL, AnswerType.NO_IN_YESNO, AnswerType.YES_IN_YESNO])
                            print([AnswerType.A_IN_ABCD, AnswerType.B_IN_ABCD, AnswerType.C_IN_ABCD, AnswerType.D_IN_ABCD])
                            raise NotImplementedError
                        image_ids.append(image_id)

                    captions = [
                        image_to_caption_mapping[image_id] for image_id in image_ids
                    ]

                    assert r"{factual_prompt}" in reward_model_prompt_untokenized

                    reward_model_prompts = []

                    for caption_list in captions:
                        caption_list = caption_list[:]
                        random.shuffle(caption_list)
                        factual_prompt = FACTUAL_PROMPT
                        for caption in caption_list:
                            factual_prompt = factual_prompt + f"  - {caption}\n"
                        reward_model_prompt_per_example = (
                            reward_model_prompt_untokenized.format(
                                factual_prompt=factual_prompt
                            )
                        )
                        reward_model_prompts.append(reward_model_prompt_per_example)
                    reward_model_prompts = self.tokenizer(
                        reward_model_prompts,
                        return_tensors="pt",
                        add_special_tokens=False,
                        padding="longest",
                    )["input_ids"]
                    reward_model_prompts = reward_model_prompts.to(
                        self.accelerator.device
                    )

                    sequences = torch.cat((sequences, reward_model_prompts), dim=1)
                    sequences = remove_pad_and_left_pad(
                        sequences,
                        pad_token_id=self.tokenizer.pad_token_id,
                    )

            clean_sequences = sequences.detach().clone()
            clean_sequences[
                clean_sequences == IMAGE_TOKEN_INDEX
            ] = self.tokenizer.eos_token_id

            text_sequences = self.tokenizer.batch_decode(
                clean_sequences,
                skip_special_tokens=False,
                clean_up_tokenization_spaces=False,
            )

            if self.accelerator.is_main_process:
                print("=" * 20)
                print(text_sequences[0].split("<unk><s> ")[-1])
                print("=" * 20)
                image_id = image_file_ids[0].item()
                # convert int into "000000xxxxxx.jpg"
                image_id = (
                    "https://s3.us-east-1.amazonaws.com/images.cocodataset.org/train2017/"
                    + str(image_id).zfill(12)
                    + ".jpg"
                )
                print(image_id)
                print("=" * 20)
            # OPTIONAL: compute the length bonus 
            non_pad_mask = responses.ne(self.tokenizer.pad_token_id)
            non_pad_seq_len = (
                non_pad_mask.sum(dim=1).float().to(self.accelerator.device)
            )
            length_bonus = non_pad_seq_len / float(self.args.response_len)

            # convert length_bonus_multiplier to the shape, type, and device of length_bonus
            length_bonus = length_bonus * length_bonus_multiplier.to(
                length_bonus.device
            ).reshape(length_bonus.shape).to(length_bonus.dtype)

            sequences_attention_mask = sequences.ne(self.tokenizer.pad_token_id)

            # Evaluate logprobs of the samples.



            rollouts_batch["length_bonus"] = length_bonus
            rollouts_batch["correct_bonus"] = correct_bonus
            sub_batch_size = self.args.reward_model_per_device_batch_size
            batch_size_per_device = rollouts_batch["responses"].shape[0]
            if sub_batch_size is None or sub_batch_size == batch_size_per_device:
                reward_outputs = self.reward_model(
                    input_ids=sequences,
                    attention_mask=sequences_attention_mask,
                    images=reward_images,
                )
                print(reward_outputs)
            else:
                assert batch_size_per_device % sub_batch_size == 0

                reward_outputs_list = []

                for sub_batch_idx in range(batch_size_per_device // sub_batch_size):
                    idx_start = sub_batch_idx * sub_batch_size
                    idx_end = (sub_batch_idx + 1) * sub_batch_size
                    sub_batch_reward_outputs = self.reward_model(
                        input_ids=sequences[idx_start:idx_end],
                        attention_mask=sequences_attention_mask[idx_start:idx_end],
                        images=reward_images[idx_start:idx_end],
                    )
                    reward_outputs_list.append(sub_batch_reward_outputs)

                reward_outputs = common_utils.merge_dict(
                    reward_outputs_list, merge_fn=torch.cat
                )
                del reward_outputs_list
                del sub_batch_reward_outputs
            # Remove the penality for sequences that did not stop properly
            # reward_outputs = self.post_reward(
            #     reward_outputs,
            #     responses,
            #     penalize_no_stop_token=self.args.penalize_no_stop_token,
            #     relative_stop_token_penalty=self.args.relative_stop_token_penalty,
            #     has_stop_token=has_stop_token,
            # )
            rollouts_batch.update(reward_outputs)
            print(f'rollouts_batch: {rollouts_batch}')

            # Shape reward with KL penalty.
            # shape_reward_outputs = self._shape_reward(
            #     rewards=rollouts_batch["rewards"],
            #     responses=rollouts_batch["responses"],
            #     logprobs=rollouts_batch["logprobs"],
            #     ref_logprobs=rollouts_batch["ref_logprobs"],
            #     length_bonus=rollouts_batch["length_bonus"],
            #     correct_bonus=rollouts_batch["correct_bonus"],
            # )
            rollouts_batch_cpu = {
                key: value for key, value in rollouts_batch.items()
            }
            rollouts.append(rollouts_batch_cpu)

        # # Items in dict need to be of same shape.
        # rollouts = common_utils.merge_dict(rollouts, merge_fn=torch.cat)

        # # Estimating advantages outside the loop gives more samples for reward normalization.
        # advantages = self._estimate_advantage(
        #     rewards=rollouts["shaped_rewards"].to(self.accelerator.device),
        #     values=rollouts["values"].to(self.accelerator.device),
        # )
        # advantages = {key: value.cpu() for key, value in advantages.items()}
        # pdb.set_trace()
        return rollouts_batch


    def post_reward(
        self,
        reward_outputs: Dict[str, torch.Tensor],
        responses: torch.Tensor,
        penalize_no_stop_token: bool,
        relative_stop_token_penalty: bool,
        has_stop_token: List[bool],
    ) -> Dict[str, torch.Tensor]:
        """Assign bad reward values to sequences which didn't stop properly."""
        if penalize_no_stop_token:
            has_stop_token = torch.tensor(has_stop_token, device=responses.device)
            rewards = reward_outputs["rewards"]
            if relative_stop_token_penalty:
                rewards = (
                    rewards + (~has_stop_token).float() * self.args.penalty_reward_value
                )
            else:
                rewards[~has_stop_token] = self.args.penalty_reward_value
            reward_outputs["rewards"] = rewards
            return reward_outputs

        if self.args.truncate_token_ids is None:
            return reward_outputs

accelerator = AlpacaAccelerator(
    log_with=training_args.report_to,
    project_dir=training_args.logging_dir,
    gradient_accumulation_steps=training_args.gradient_accumulation_steps,
    even_batches=True,  # Make sure the batch size on each device is the same.
    split_batches=False,  # Don't break a batch into smaller chunks.
    step_scheduler_with_optimizer=False,  # Untie optimizer and scheduler step.
    # Value model might not use all parameters (e.g., lm-head) in the forward pass.
    kwargs_handlers=[
        DistributedDataParallelKwargs(
            find_unused_parameters=training_args.ddp_find_unused_parameters,
        )
    ],
)
attribute_model = GetAttributeModel(rm_model, tokenizer, accelerator, training_args)
attribute_model.training_args.rollout_per_device_batch_size = 4
train_dataloader = attribute_model.get_train_dataloader()
for i in range(1):
    rollout = attribute_model.step(train_dataloader, i)
    print(rollout.keys())
    # Saves the attributes to the prompt

    

/home/ubuntu/mambaforge-pypy3/envs/llavav2/lib/python3.10/site-packages/accelerate/accelerator.py:371: UserWarning: `log_with=[]` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")
Train dataset size: 72056


rollout:   0%|          | 0/1 [00:00<?, ?it/s]

A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: </s> 
What issue does the sign address, and why is it important? ASSISTANT: The sign in the image addresses the issue of hazardous road conditions, specifically warning cyclists to be cautious in the area due to potential holes or uneven surfaces that may cause them to fall off their bicycles. It is important as it serves to alert cyclists to potential dangers so they can be prepared, slow down, and take necessary precautions to ensure their safety. This indication can help prevent accidents and injuries, contributing to a safer environment for both cyclists and other road<s>A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: 
 ASSISTANT: The sign in the image addresses the issue of hazardous road conditions, specificall

rollout:   0%|          | 0/1 [00:04<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 11.71 GiB (GPU 0; 79.15 GiB total capacity; 52.41 GiB already allocated; 8.61 GiB free; 69.12 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [62]:
torch.softmax(rollout['rewards'])

TypeError: softmax() received an invalid combination of arguments - got (Tensor), but expected one of:
 * (Tensor input, int dim, torch.dtype dtype, *, Tensor out)
 * (Tensor input, name dim, *, torch.dtype dtype)
